In [1]:
import numpy as np
import cvxpy as cp
import random

In [8]:
def generate_support(nmaps, nqubits, nmaps_per_qubit, qubit_occupancies):
    mat = cp.Variable(shape=(nmaps,nqubits), boolean = True)
    constraints = []
    col_sums = cp.sum(mat, axis=0, keepdims=True)
    constraints.append(col_sums >= 1)
    row_sums = cp.sum(mat, axis=1)
    constraints.append(row_sums == qubit_occupancies)
    objective = cp.Minimize(cp.norm(col_sums-nmaps_per_qubit,"inf"))
    problem = cp.Problem(objective,constraints)
    problem.solve()
    if problem.status not in ["infeasible", "unbounded"]:
        support = [tuple(np.nonzero(row)[0]) for row in mat.value]
        print("Optimization value = {}".format(problem.value))
        print("Occupancy matrix = {}".format(mat.value))
        print("Number of maps per qubit : {}".format(np.sum(mat.value,axis=0)))
        print("Support : {}".format(support))
    else:
        print("Qubit allocation to maps infeasible.")

In [9]:
nmaps = 4
nqubits = 7
nmaps_per_qubit = max(0.1*nmaps,1)
qubit_occupancies = np.array([ random.randint(1, nqubits//2) for __ in range(nmaps)])
print("Qubit occupancies : {}".format(qubit_occupancies))
generate_support(nmaps, nqubits, nmaps_per_qubit, qubit_occupancies)

Qubit occupancies : [2 2 2 1]
Optimization value = 0.0
Occupancy matrix = [[1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]
Number of maps per qubit : [1. 1. 1. 1. 1. 1. 1.]
Support : [(0, 1), (2, 4), (3, 5), (6,)]
